In [1]:
from lkae.utils.data_loading import AuredDataset, RumorWithEvidence, eng_combined_jsonl
import json 

with open('config.json', 'r') as file:
    config = json.load(file)

ds = AuredDataset(eng_combined_jsonl, **config)

sample: RumorWithEvidence = ds[1]
# print(json.dumps(sample, indent=2))

In [2]:
# we fake similarity scores for RQ2

import json
from copy import deepcopy
from lkae.utils.data_loading import AuthorityPost

rq2_dataset = deepcopy(ds)

for i, item in enumerate(rq2_dataset):
    item['retrieved_evidence'] = []

    for author_account, tweet_id, tweet_text, _, _ in item['evidence']:
        item['retrieved_evidence'] += [
            AuthorityPost(
                author_account, #'author_account'
                tweet_id, #'authority_tweet_id'
                tweet_text, #'doc_text'
                1, #'rank'
                1, #'score'
            )
            
        ]

    rq2_dataset[i] = item
# print(json.dumps(rq2_dataset[0], indent=2))

In [5]:
from lkae.verification.verify import run_verifier_on_dataset, Judge
from lkae.verification.models.ollama import OllamaVerifier
import json
import os

j = Judge()

with open('config.json', 'r') as file:
    config = json.load(file)

verifier = OllamaVerifier()
res_jsons = run_verifier_on_dataset(dataset=rq2_dataset[0:4], verifier=verifier, judge=j, blind=config['blind_run'])
print(json.dumps(res_jsons, indent=2))

-----total token usage for verification-----
total tokens:	0
prompt tokens:	0
completion tokens:	0
price estimate:	$0.0
[
  {
    "id": "AuRED_142",
    "label": "REFUTES",
    "claim": "Naturalization decree in preparation: Lebanese passports for sale !",
    "predicted_label": "REFUTES",
    "predicted_evidence": [
      [
        "https://twitter.com/LBpresidency",
        "1555986659279360001",
        "Statement from Authority Account 'LBpresidency': ''The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4 000 people and recalls that it had denied yesterday the false information published by the French magazine 'Liberation' about the same fabricated news ''",
        0.9
      ],
      [
        "https://twitter.com/LBpresidency",
        "1555424541509386240",
        "Statement from Authority Account 'LBpresidency': ''The Information Office of the Presidency of the Republic: 